# Overview

We need to refine the way users specify colors, to simplify the public API and eliminate inconsistencies.  The challenge is to handle all of the ways in which a user might want to specify per-series / per-datum colors, while avoiding potential ambiguities.

**Use-cases**

* Specify one explicit color for everything.
        toyplot.plot(x, y, color="red")
* Specify an explicit color per series.
        toyplot.plot(x, y, color=["red", "cinnamon", "cardinal"])
* Specify a palette for the series.
        toyplot.plot(x, y, color=toyplot.color.brewer("reds"))
* Specify a colormap for the series.
        toyplot.plot(x, y, color=toyplot.color.LinearMap(toyplot.color.Palette(["red", "green", "blue"])))
* Specify per-datum scalar values and have them mapped.
        toyplot.plot(x, y, color=temperature)
* Specify per-datum scalar values and have them mapped, but override a few "special" values.
        colors = toyplot.color.broadcast(temperature)
        colors[numpy.argsort(temperature)[0]] = "blue"
        colors[numpy.argsort(temperature)[-1]] = "red"
        toyplot.plot(x, y, color=colors)
* Specify explicit per-datum color values for everything (i.e. map them yourself).
        toyplot.plot(x, y, color=mycolors)

**API**

* <strike>(values) - map the values with the default colormap.</strike>
* (colormap) - Use the given colormap to map series $[0, N)$.
    * What if there's only one series?  Users may expect it to map the datums.
* (palette) - Use the palette with a linear colormap to map series $[0, N)$.
    * What if there's only one series?  Users may expect it to map the datums.
* <strike>(values, colormap) - map values with the given colormap.</strike>
* <strike>(values, palette) - map values with a linear colormap and the given palette.</strike>

toyplot.color.broadcast(...) always returns a numpy color array (array with dtype == toyplot.color.dtype), with the given shape.

**Values**

* <strike>"value" - a single CSS color value.</strike>
* <strike>(r, g, b) - a single RGB color value.</strike>
* <strike>(r, g, b, a) - a single RGBA color value.</strike>
* <strike>toyplot.color.rgb() - a single RGB color value.</strike>
* <strike>toyplot.color.rgba() - a single RGBA color value.</strike>
* <strike>[value1, value2, ...] - 1D heterogeneous collection of CSS and tuple color values.</strike>
    * We don't allow scalar values in this case, because their domain would be ambiguous.
    * Have to be careful with the implementation here - we don't want a list of nothing-but-tuples to be misinterpreted as a 2D numpy array of scalars.
* <strike>numpy string array with shape $M$ - 1D collection of CSS color values.</strike>
* numpy string array with shape $M \times N$ - 2D collection of CSS color values.
* <strike>numpy numeric array with shape $M$ - 1D collection of scalar values for mapping.</strike>
* <strike>numpy numeric array with shape $M \times N$ - 2D collection of scalar values for mapping.</strike>
* <strike>numpy color array with shape $M$ - 1D collection of color values.</strike>
* numpy color array with shape $M \times N$ - 2D collection of color values.

In [1]:
import numpy
x = numpy.linspace(0, 1)
y = numpy.column_stack((x ** 2, x ** 3))
numpy.random.seed(1234)
temperature = numpy.random.uniform(size=y.shape)

In [2]:
import toyplot

In [3]:
toyplot.scatterplot(x, y, width=300);

In [4]:
toyplot.scatterplot(x, y, color="red", width=300);

In [5]:
toyplot.scatterplot(x, y, color=["red", "blue"], width=300);

In [6]:
toyplot.scatterplot(x, y, fill=temperature, width=300);

In [7]:
toyplot.scatterplot(x, y, fill=temperature, fill_palette=toyplot.color.brewer("BlueRed"), width=300);

In [8]:
toyplot.scatterplot(x, y, fill=(temperature, toyplot.color.brewer("BlueRed")), width=300);

In [9]:
toyplot.scatterplot(x, y, fill=(temperature, toyplot.color.LinearMap()), width=300);

In [10]:
colors = toyplot.color.broadcast((temperature, toyplot.color.brewer("Greys")), y.shape)
colors[20:30, 1] = toyplot.color.css("yellow")
colors.flat[numpy.argmin(temperature)] = toyplot.color.css("blue")
colors.flat[numpy.argmax(temperature)] = toyplot.color.css("red")
toyplot.scatterplot(x, y, fill=colors, width=300);

In [11]:
x = numpy.linspace(0, 1)
y = x ** 2

In [12]:
toyplot.bars(y, fill=(x, toyplot.color.brewer("Reds")), width=300)

(<toyplot.canvas.Canvas at 0x110b61a90>,
 <toyplot.mark.BarMagnitudes at 0x110b4ded0>)

In [13]:
toyplot.fill(x, numpy.column_stack((y, y/2, y/3, y/4, y/5)), baseline="symmetric", fill=(numpy.arange(5), toyplot.color.brewer("Reds")), width=300)

(<toyplot.canvas.Canvas at 0x110bc1c50>,
 <toyplot.mark.FillMagnitudes at 0x110bc4dd0>)

In [14]:
toyplot.plot(x, numpy.column_stack((y, y/2,y/3,y/4,y/5)), color=numpy.arange(5), palette=toyplot.color.brewer("Reds"), width=300)

(<toyplot.canvas.Canvas at 0x110c088d0>,
 <toyplot.mark.Plot at 0x110c18f50>)

In [15]:
toyplot.scatterplot(x, numpy.column_stack((y, y/2, y/3, y/4, y/5)), color=(numpy.arange(5), toyplot.color.brewer("Reds")), width=300)

(<toyplot.canvas.Canvas at 0x110bca090>,
 <toyplot.mark.Plot at 0x110ba2210>)

In [16]:
canvas = toyplot.Canvas(width=300)
axes = canvas.axes()
axes.text(x, y, y, fill=(x, toyplot.color.brewer("Reds")))

In [17]:
canvas = toyplot.Canvas(width=300)
axes = canvas.axes()
axes.hlines(x, stroke=(x, toyplot.color.brewer("Reds")))

In [18]:
canvas = toyplot.Canvas(width=300)
axes = canvas.axes()
axes.vlines(x, stroke=(x, toyplot.color.brewer("Reds")))